In [ ]:
import numpy as np
import cv2
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import os
from numpy import linalg as LA
from numpy.linalg import inv

## Load Image

In [ ]:
def LoadImg(path):
    img = cv2.imread(path)
    return img.reshape(imgSize, 3)

imgL = 100
imgSize = imgL*imgL
imgs = []
for i in range(2):
    imgs.append(LoadImg(f"./data/image{i+1}.png"))

## Kernel
$$K(x,x') = e^{-\gamma_s||S(x)-S(x')||^2} *  e^{-\gamma_c||C(x)-C(x')||^2}$$

In [ ]:
def Kernel(img):
    K = np.zeros((imgSize,imgSize))
    # Spetial distance
    pos = np.array([(p // imgL, p % imgL) for p in range(imgSize)])
    Sd = cdist(pos, pos, metric='sqeuclidean')
    # Color distance
    Cd = cdist(img, img, metric='sqeuclidean')
    kernel = np.exp(-Gamma_s * Sd) * np.exp(-Gamma_c * Cd)
    return kernel

## Eigen Decomposition

In [ ]:
def EigenDecomp(L):
    eigenvalues, eigenvectors = LA.eig(L)
    sorted_idx = np.argsort(eigenvalues)
    return eigenvectors[:, sorted_idx[1:(K + 1)]].real

In [ ]:
def Normalized(L,D):
    D_inv_sqrt = np.diag(1 / np.sqrt(np.diag(D)))
    L_sym = D_inv_sqrt @ L @ D_inv_sqrt
    T = EigenDecomp(L_sym)

    H = D_inv_sqrt @ T
    return H

## Kmeans clustering
1. Initialize k cluster center$\mu_{k}$
2. Estep: fixed mean, assigned $r_{nk}$ to the closest cluster
3. Mstep: fixed $r_{nk}$, calculate new center
4. Repeat EM until $\mu_{k}$ not changed

In [ ]:
def Init(X):
    # Initial Centers, save value
    # (1) random initialize
    M = []
    R = np.zeros((imgSize, K))
    if Mode == 0:
        M_idx = np.random.choice(imgSize, size=K, replace=False)
        M = X[M_idx]
        R[M_idx]=1

    # (2) Kmeans++
    else:
        M = []
        m0_idx = np.random.choice(imgSize)
        M.append(X[m0_idx])
        R[m0_idx,0]=1
        for k in range(1, K):
            min_d = []
            for x in X:
                mean_d = [np.linalg.norm(x - m) for m in M]
                min_dist = min(mean_d)
                min_d.append(min_dist**2)

            min_d = np.array(min_d)
            # Roulette selection
            p = min_d/np.sum(min_d)
            cumulative_p = np.cumsum(p)
            r = np.random.rand()
            next_c = np.where(cumulative_p >= r)[0][0]

            M.append(X[next_c])
            R[next_c,k]=1
    return M, R


In [ ]:
def NewCluster(X,M):
    R_new = np.zeros((imgSize, K))
    for n in range (imgSize):
        d = np.sum((X[n] - M) ** 2, axis=1)
        c = np.argmin(d)
        R_new[n,c] = 1
    return R_new

In [ ]:
def NewMean(X,R):
    M_new = []
    for k in range(K):
        Ck = X[R[:, k] == 1]
        m = np.mean(Ck, axis=0)
        M_new.append(m)
    return np.array(M_new)


In [ ]:
def Kmeans(X):
    M_last, R= Init(X)
    diff = imgSize
    iter_=1

    while diff>0:
        # Estep
        R = NewCluster(X, M_last)

        # Mstep
        M = NewMean(X,R)

        diff = np.sum(np.abs(M-M_last))
        print(f"Iteration:{iter_}, Difference: {diff}")
        VizAll(img, U, R, iter_, f"{CutMap[Cut]} Spectral, Clusters:{K}, Init:{ModeMap[Mode]}")

        if diff==0:
            break
        M_last = M
        iter_+=1



## Viz

In [ ]:
Color = [[0, 0, 255],    #blue
         [255, 165, 0],  #orange
         [0, 255, 0],    #green
         [255, 0, 0]]   #red

ColorE = ['blue','orange', 'green','red']

def VizAll(img, U, R, iter_, title):
    # Clustered img
    img_iter = np.zeros((imgL, imgL, 3), dtype=np.uint8)
    cluster = np.argmax(R, axis=1)
    for n in range(imgSize):
        img_iter[n // imgL][n % imgL] = Color[cluster[n]]

    # Eigenspace scatter data
    px, py, pz = [], [], []
    for _ in range(K):
        px.append([])
        py.append([])
        if K == 3:
            pz.append([])

    for n in range(imgSize):
        px[cluster[n]].append(U[n][0])
        py[cluster[n]].append(U[n][1])
        if K == 3:
            pz[cluster[n]].append(U[n][2])



    if K==2:
        fig = plt.figure(figsize=(9, 3.5))
        fig.suptitle(title, fontsize=14, y=0.96)
    else:
        fig = plt.figure(figsize=(9, 4))
        fig.suptitle(title, fontsize=14, y=0.98)

    ax1 = fig.add_subplot(1,3,1)
    img_ = cv2.cvtColor(img.reshape(imgL, imgL, 3), cv2.COLOR_RGB2BGR)
    ax1.imshow(img_)
    ax1.axis('off')

    ax2 = fig.add_subplot(1,3,2)
    ax2.imshow(img_iter)
    ax2.axis('off')
    ax2.set_title(f"Iteration: {iter_}")

    if K == 2:
        ax3 = fig.add_subplot(1,3,3)
        for k in range(K):
            ax3.scatter(px[k], py[k], c=ColorE[k])
        ax3.set_title("Eigenspace Clustering")
    elif K == 3:
        ax3 = fig.add_subplot(1,3,3, projection='3d')
        for k in range(K):
            ax3.scatter(px[k], py[k], pz[k], c=ColorE[k])
        ax3.set_title("Eigenspace Clustering")

    plt.tight_layout()
    plt.show()
    fig.savefig(f'{Outpath}/{iter_}.jpg')


## Spectral Clustering
Ratio cut:
1. Use kernel to construct similarity graph, W is the weighted adjency matrix
2. Compute  $L = D-W$
3. Find the eigenvector $u_1, u_2,...,u_k$ of $L $, $U = [u_1 u_2...u_k]$
5. $y_i$ is the i-th row of U
6. cluster points $y_i$ with kmeans
7. Output: $A_1, A_2,...,A_k, A_i = \{j|y_j\in Ci\}$

Normalizsd cut:
1. Use kernel to construct similarity graph, W is the weighted adjency matrix
2. Compute  $L_{sym} = I - D^{\frac{1}{2}}WD^{\frac{1}{2}}$
3. Find the eigenvector $u_1, u_2,...,u_k$ of $L_{sym}$, $U = [u_1 u_2...u_k]$
4. Calculate T that $ t_{ij} = \frac{u_{ij}}{\sqrt{\sum_k u_{ik}^2}}$
5. $y_i$ is the i-th row of T
6. cluster points $y_i$ with kmeans
7. Output: $A_1, A_2,...,A_k, A_i = \{j|y_j\in Ci\}$

In [ ]:
Ks = [2,3]
Modes = [0, 1]
ModeMap = {0:"random", 1:"kmeans++"}
Cuts = [0, 1]
CutMap = {0:"Ratio Cut", 1:"Normalized Cut"}
CutMap2 = {0:"Ratio", 1:"Normalized"}

for img_idx in range(2):
    img = imgs[img_idx]
    for K in Ks:
        # Construct plot
        Gamma_s = 0.0001
        Gamma_c = 0.001

        W = Kernel(img)
        D = np.diag(np.sum(W,axis = 1))
        L = D - W

        for Cut in Cuts:
            if Cut == 0:
                    U = EigenDecomp(L)
            else:
                    U = Normalized(L,D)
            Outpath2 = f"./result/Spectral/U"
            try:
                os.mkdir(Outpath2)
            except:
                pass
            np.save(os.path.join(Outpath2, f"Image{img_idx+1}_{K}clusters_{CutMap2[Cut]}.npy"), U)

            for Mode in Modes:
                Outpath = f"./result/Spectral/Image{img_idx+1}_{K}clusters_{ModeMap[Mode]}_{CutMap[Cut]}"
                try:
                    os.mkdir(Outpath)
                except:
                    pass
                Kmeans(U)
